In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import cv2
from albumentations import Rotate
from scipy import ndimage
import imutils
from skimage import data
from skimage import color
from skimage.filters import meijering, sato, frangi, hessian
from skimage.morphology import medial_axis, skeletonize, thin, remove_small_objects
import rdp
import math
from tqdm import tqdm
import sys
sys.path.append('../Scripts/')
from segmentation_utils import *
from metrics import *
from image_utils import *

In [2]:
def calculate(path_list):
    df = pd.DataFrame(columns = ['img_id','tort', 'dm', 'ip', 'tp', 'sd'])
    all_metrics = []
    all_img_ids = []
    for i, path in tqdm(enumerate(path_list)):
        img_orig = cv2.imread(path, 0)
        seg_img = segment(img_orig)
        all_points = get_points(seg_img)
        all_unique = get_all_unique(all_points)
        points = get_turning_points(seg_img,tolerance=2,min_angle=np.pi*0.15)
        vessel_metrics = []
        for j in range(len(all_unique)):
            tort,dm,ip,tp,sd = tortousity(all_unique[j])
            vessel_metrics.append([tort,dm,ip,tp,sd])
            all_img_ids.append(path.split('/')[-1])
        all_metrics.append(np.array(vessel_metrics))
    df.img_id = all_img_ids
    df.iloc[:, 1:6] = np.concatenate(all_metrics)
    return df

In [3]:
img_paths1 = sorted(glob.glob('../Data/SPIE_2020_Retinal_Tortuosity/Emmetropia/*')) #Original Folder
img_paths2 = sorted(glob.glob('../Data/SPIE_2020_Retinal_Tortuosity/Myopia/*')) #Original Folder
img_paths3 = sorted(glob.glob('../Data/SPIE_2020_Retinal_Tortuosity/Hypermetropia/*')) #Original Folder
img_paths = img_paths1 + img_paths2 + img_paths3

df = calculate(img_paths)

2it [09:35, 304.31s/it]../Scripts/metrics.py:287: RuntimeWarning: invalid value encountered in double_scalars
  dm+=al/cl-1
../Scripts/metrics.py:292: RuntimeWarning: invalid value encountered in double_scalars
  dm+=al/cl-1
65it [12:08:18, 672.29s/it] 


In [35]:
img_paths1 = sorted(os.listdir('../Data/SPIE_2020_Retinal_Tortuosity/Emmetropia/')) #Original Folder
img_paths2 = sorted(os.listdir('../Data/SPIE_2020_Retinal_Tortuosity/Myopia/')) #Original Folder
img_paths3 = sorted(os.listdir('../Data/SPIE_2020_Retinal_Tortuosity/Hypermetropia/')) #Original Folder
class_mapping1 = { i : 'Emmetropia' for i in img_paths1 }
class_mapping2 = { i : 'Myopia' for i in img_paths2 }
class_mapping3 = { i : 'Hypermetropia' for i in img_paths3 }

In [36]:
class_mapping = {**class_mapping1, **class_mapping2, **class_mapping3}

In [37]:
df['Class'] = ''
for i in range(len(df)):
    df.Class.iloc[i] = class_mapping[df.img_id.iloc[i]]

/home/arnav0400/.local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [39]:
df.Class.value_counts()

Hypermetropia    1571
Emmetropia       1365
Myopia           1364
Name: Class, dtype: int64

In [40]:
df.to_csv('../Data/complete_data.csv', index = False)

In [49]:
df.groupby.sd.mean()

20.202940918447442

In [3]:
df = pd.read_csv('../Data/complete_data.csv')

In [24]:
df.groupby(['Class', 'img_id']).tp.describe()

count  \
Class      img_id                                                      
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  110.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...   45.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...   41.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...   41.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...   75.0   
...                                                              ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...   51.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...   57.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  101.0   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...   63.0   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...   41.0   

                                                                   mean  \
Class      img_id                                                         
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  0.645455   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.844444   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.682927   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.878049   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.853333   
...                                                                 ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  0.725490   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.754386   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.960396   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  0.746032   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.682927   

                                                                    std  min  \
Class      img_id                                                              
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  1.009917  0.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  1.106911  0.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.878608  0.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.899864  0.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  1.074185  0.0   
...                                                                 ...  ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  0.776619  0.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.911841  0.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  1.355882  0.0   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  1.046781  0.0   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.906615  0.0   

                                                               25%  50%  75%  \
Class      img_id                                                              
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  0.0  0.0  1.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.0  1.0  1.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.0  0.0  1.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.0  1.0  2.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.0  0.0  1.5   
...                                                            ...  ...  ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  0.0  1.0  1.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.0  0.0  1.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.0  1.0  1.0   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  0.0  0.0  1.0   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.0  0.0  1.0   

                                                               max  

In [23]:
df.groupby(['Class', 'img_id']).sd.describe()

count  \
Class      img_id                                                      
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  110.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...   45.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...   41.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...   41.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...   75.0   
...                                                              ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...   51.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...   57.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  101.0   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...   63.0   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...   41.0   

                                                                    mean  \
Class      img_id                                                          
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  15.206206   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  25.367799   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  20.374835   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  26.898644   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  15.150243   
...                                                                  ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  18.546589   
           Divya_K__1204142_19951025_Female_Angiography 6x...  20.581936   
           Divya_K__1204142_19951025_Female_Angiography 6x...  20.341945   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  15.154847   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  16.923691   

                                                                     std  min  \
Class      img_id                                                               
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  22.451453  0.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  26.452583  0.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  26.988165  0.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  28.551644  0.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  20.804021  0.0   
...                                                                  ...  ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  25.504737  0.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  26.268424  0.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  25.736809  0.0   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  22.006126  0.0   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  22.402991  0.0   

                                                                    25%  \
Class      img_id                                                         
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  0.572556   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.659386   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.203483   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  4.016716   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.906280   
...                                                                 ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  1.120907   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.503198   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.326347   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  0.493697   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.499401   

                                                                     50%  \
Class      img_id              

In [22]:
df.groupby(['Class', 'img_id']).dm.describe()

count  \
Class      img_id                                                      
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  110.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...   45.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...   39.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...   41.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...   75.0   
...                                                              ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...   51.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...   56.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  101.0   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...   63.0   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...   41.0   

                                                                   mean  \
Class      img_id                                                         
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  0.279250   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.351974   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.230928   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.321534   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.317127   
...                                                                 ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  0.096258   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.252024   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.377869   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  0.287185   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.177935   

                                                                    std  \
Class      img_id                                                         
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  0.496036   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.569173   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.416714   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.519190   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.528280   
...                                                                 ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  0.238774   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.424084   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.618974   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  0.524217   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.344474   

                                                                        min  \
Class      img_id                                                             
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  0.000000e+00   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.000000e+00   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.000000e+00   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.000000e+00   
           1204144_Divya_A__1204144_19940912_Female_Angiog... -1.820766e-14   
...                                                                     ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  0.000000e+00   
           Divya_K__1204142_19951025_Female_Angiography 6x... -2.997602e-15   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.000000e+00   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  0.000000e+00   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.000000e+00   

                                                                    25%  \
Class      img_id                                                      

In [21]:
df.groupby(['Class', 'img_id']).tort.describe()

count  \
Class      img_id                                                      
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  110.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...   45.0   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...   39.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...   41.0   
           1204144_Divya_A__1204144_19940912_Female_Angiog...   75.0   
...                                                              ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...   51.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...   56.0   
           Divya_K__1204142_19951025_Female_Angiography 6x...  101.0   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...   63.0   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...   41.0   

                                                                   mean  \
Class      img_id                                                         
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  0.008675   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.012551   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.011054   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.005435   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.008054   
...                                                                 ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  0.004714   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.008368   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.013729   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  0.011446   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.005216   

                                                                    std  \
Class      img_id                                                         
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  0.022221   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.030521   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.032504   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.009995   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.018354   
...                                                                 ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  0.020072   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.024139   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.033183   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  0.030620   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.016010   

                                                                        min  \
Class      img_id                                                             
Emmetropia 1193671_Clear_jandyal_sourabh__1193671_19951205...  0.000000e+00   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.000000e+00   
           1204143_Sureandhiran_R__1204143_19960529_Male_A...  0.000000e+00   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.000000e+00   
           1204144_Divya_A__1204144_19940912_Female_Angiog...  0.000000e+00   
...                                                                     ...   
Myopia     Ashika_G__1204321_19960716_Female_Angiography 6...  0.000000e+00   
           Divya_K__1204142_19951025_Female_Angiography 6x... -7.060031e-16   
           Divya_K__1204142_19951025_Female_Angiography 6x...  0.000000e+00   
           GANESH_C__333542_19690708_Male_Angiography 6x6 ...  0.000000e+00   
           Gnanapoonkodi_B__4259046_19960225_Female_Angiog...  0.000000e+00   

                                                               25%       50%  \
Class      img_id                                                 